In [1]:
# 1. Install necessary libraries
!pip install langchain langchain-openai langchain-core -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.5 MB/s eta 0:00:00


In [7]:
!pip install langchain-neo4j -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.0.5 which is incompatible.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
langchain 0.3.27 requires

In [2]:
'''
WARNING: Package(s) not found: langchain-community
Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page:
Author:
Author-email:
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-openai, langchain-text-splitters
---
Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page:
Author:
Author-email:
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by:
'''

'\nWARNING: Package(s) not found: langchain-community\nName: langchain-core\nVersion: 0.3.79\nSummary: Building applications with LLMs through composability\nHome-page:\nAuthor:\nAuthor-email:\nLicense: MIT\nLocation: /usr/local/lib/python3.12/dist-packages\nRequires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions\nRequired-by: langchain, langchain-openai, langchain-text-splitters\n---\nName: langchain\nVersion: 0.3.27\nSummary: Building applications with LLMs through composability\nHome-page:\nAuthor:\nAuthor-email:\nLicense: MIT\nLocation: /usr/local/lib/python3.12/dist-packages\nRequires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy\nRequired-by:\n'

In [3]:
!pip show langchain-community langchain-core langchain

Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-openai, langchain-text-splitters
---
Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [4]:
import langchain
print(langchain.__version__)

0.3.27


In [28]:
def load_properties(path):
    data = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.split("=", 1)
                data[k.strip()] = v.strip()
    return data



In [30]:
props = load_properties("openai_key.txt")

os.environ["NEO4J_URI"] = props["NEO4J_URI"]
os.environ["OPENAI_KEY"] = props["OPENAI_KEY"]
os.environ["NEO4J_USERNAME"] = props["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"] = props["NEO4J_PASSWORD"]
os.environ["NEO4J_DATABASE"] = props["NEO4J_DATABASE"]

print(os.environ["NEO4J_URI"])

neo4j+s://8e403408.databases.neo4j.io


In [61]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent,create_tool_calling_agent
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_neo4j import Neo4jChatMessageHistory, Neo4jGraph
from langchain_core.messages import HumanMessage, AIMessage
from uuid import uuid4

In [32]:
# ---------------------------
# Session ID
# ---------------------------
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: 7478b78e-7344-4930-af87-0060beb6fac3


In [33]:
# instantiating the base llm object

#from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

myllmObject = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    api_key=os.getenv("OPENAI_KEY")
  )

In [100]:
# Test connectivitity with Neo4j Database
uri=os.environ["NEO4J_URI"]
user = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]

driver = GraphDatabase.driver(uri, auth=(user, password))
# Simple test query
with driver.session() as session:
    result = session.run("RETURN 1 AS test")
    print(result.single())  # Should print: Record(test=1)

<Record test=1>


In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("You are a Neo4j expert helping the user create Cypher queries.{query}?")

parser = StrOutputParser()

cyphersupportchain = prompt | myllmObject | parser



In [94]:
from langchain.tools import tool
from neo4j import GraphDatabase
# Assuming countrycapitalchain.run is your function to get capital of a country
@tool
def cyphersupport_tool(query: str) -> str:
    """Useful for when you need to query. graph database"""
    driver = GraphDatabase.driver(os.environ["NEO4J_URI"],
                                auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))

    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]



In [88]:
# When giving tools to an LLM, pass as a list
tools = [cyphersupport_tool]

In [89]:
# This structure is critical for passing history and scratchpad
agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",  "You are a Neo4j expert helping the user create Cypher queries using tool 'cyphersupport_tool'. Execute the query and show result "),
        # This is where the Agent Executor injects past messages (memory)
        MessagesPlaceholder(variable_name="chat_history"),
        # This is the new user input
        ("human", "{input}"),
        # This is where the Agent Executor injects the LLM's thoughts and tool observations
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [106]:
# Assemble the Agent (LCEL / Pipeline) ---

# A. Create the Agent: The runnable that decides what action to take (LLM + Tools + Prompt)
agent = create_tool_calling_agent(myllmObject, tools, agent_prompt)

# B. Create the Executor: The runtime that manages the loop (Agent + Execution logic)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=False # See the reliable tool-calling JSON flow
)

In [96]:
chat_history = [
    HumanMessage(content="Hello! My name is John."),
    AIMessage(content="Hello John! How can I help you today?"),
]

In [107]:
result_1 = agent_executor.invoke({
    "input": "find total number of movie nodes",
    "chat_history": chat_history # Pass the history explicitly
})
print(f"\n🤖 AI Response 1: {result_1['output']}")


🤖 AI Response 1: The total number of movie nodes in the database is 93.


In [84]:
result_1

{'input': 'find total number of movie nodes',
 'chat_history': [HumanMessage(content='Hello! My name is John.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I help you today?', additional_kwargs={}, response_metadata={})],
 'output': 'The total number of movie nodes in the database is 10.'}

In [109]:
result_2 = agent_executor.invoke({
    "input": "find top 10 movies where Chris Tucker worked",
    "chat_history": chat_history # Pass the history explicitly
})
print(f"\n🤖 AI Response 1: {result_2['output']}")


🤖 AI Response 1: The top 10 movies where Chris Tucker worked are:
1. Dead Presidents (Released: 1995-10-06)
2. Friday (Released: 1995-04-26)


In [112]:
# Get all messages
for msg in memory.chat_memory.messages:
    print(msg)

content='find total number of movie nodes' additional_kwargs={} response_metadata={}
content='The total number of movie nodes in the database is 93.' additional_kwargs={} response_metadata={}
content='find total number of movie nodes' additional_kwargs={} response_metadata={}
content='The total number of movie nodes in the database is 93.' additional_kwargs={} response_metadata={}
content='find atop 10 movies where Chris Tucker worked' additional_kwargs={} response_metadata={}
content='The top 10 movies where Chris Tucker worked are:\n1. Dead Presidents (Released: 1995-10-06)\n2. Friday (Released: 1995-04-26)' additional_kwargs={} response_metadata={}
content='find top 10 movies where Chris Tucker worked' additional_kwargs={} response_metadata={}
content='The top 10 movies where Chris Tucker worked are:\n1. Dead Presidents (Released: 1995-10-06)\n2. Friday (Released: 1995-04-26)' additional_kwargs={} response_metadata={}


Alternative way

In [113]:
# ---------------------------
# 1️⃣ Session ID
# ---------------------------
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: c7ec68e2-ee94-48ce-a4d1-b85aac26943c


In [114]:
# ---------------------------
# 2️⃣ LLM Setup
# ---------------------------
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_KEY")  # Make sure your key is set
)


In [115]:
# ---------------------------
# 3️⃣ Neo4j Graph Connection
# ---------------------------
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

In [131]:
# ---------------------------
# 4️⃣ Local Prompt Template
# ---------------------------
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Neo4j expert helping the user create Cypher queries. "
               "You have access to the following tools:\n{tools}\n"
               "Use the following format:\n\n"
               "Question: the input question you must answer\n"
               "Thought: you should always think about what to do\n"
               "Action: the action to take, should be one of [{tool_names}]\n"
               "Action Input: the input to the action\n"
               "Observation: the result of the action\n"
               "... (this Thought/Action/Action Input/Observation can repeat N times)\n"
               "Thought: I now know the final answer\n"
               "Final Answer: the final answer to the original input question\n"
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [132]:
# ---------------------------
# 5️⃣ Cypher Chat Pipeline
# ---------------------------
cypher_chat = agent_prompt | llm | StrOutputParser()

In [133]:
# Function to get memory for chat history
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [134]:
# ---------------------------
# 6️⃣ Tools
# ---------------------------
tools = [
    Tool.from_function(
        name="Cypher Support",
        description="Use this to help the user write Cypher queries.",
        func=cypher_chat.invoke
    )
]

In [135]:

# ---------------------------
# 7️⃣ Agent Setup
# ---------------------------
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Wrap with history so chat is remembered per session
cypher_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [136]:

# ---------------------------
# 8️⃣ Run interactive loop
# ---------------------------
print("Type 'exit' to quit.")
while (q := input("> ")) != "exit":
    response = cypher_agent.invoke(
        {"input": q},
        {"configurable": {"session_id": SESSION_ID}}
    )
    print(response["output"])


Type 'exit' to quit.
> total no of movie nodes


ValueError: variable agent_scratchpad should be a list of base messages, got  of type <class 'str'>